In [78]:
# from google.colab import drive
# drive.mount('/content/drive')

In [79]:
import numpy as np
import pylab as pl
import pandas as pd
import requests
from io import BytesIO

In [80]:
key='1ud8n0zzmJgrV6re45F9gsH2I9uEmSGS_oYHaTXx9E88'

In [81]:
def extract(key):
    link = 'https://docs.google.com/spreadsheet/ccc?key=' + key + '&output=csv'

    r = requests.get(url=link)
    data = r.content

    df = pd.read_table(BytesIO(data),header=None,sep=',')

    return df

In [82]:
# df = extract(key)
df = pd.read_excel('Pórtico 1.xlsx', header=None)
df

,0,1,2,3,4,5
0,3,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,1.0,1.0,1.0
2,2,0.0,6.0,0.0,0.0,0.0
3,3,4.0,6.0,1.0,1.0,1.0
4,2,NaN,NaN,NaN,NaN,NaN
5,1,1.0,2.0,1.0,1.0,1.0
6,2,2.0,3.0,1.0,1.0,1.0
7,2,NaN,NaN,NaN,NaN,NaN
8,2,10.0,0.0,NaN,NaN,NaN
9,2,0.0,-6.0,NaN,NaN,NaN


In [83]:
class node:
	def __init__(self, x, y, constraints):
		self.x = x
		self.y = y
		self.constraints = constraints

	def __str__(self):
		return f'Node - x: {self.x}, y: {self.y}, constraints: {self.constraints}'

In [84]:
class element:
	def __init__(self, node1, node2, E, A, I):
		self.node1 = node1
		self.node2 = node2
		self.L = np.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)
		self.E = E
		self.A = A
		self.I = I

	def __str__(self):
		return f'Element - node1: {self.node1}, node2: {self.node2}, L: {self.L}, E: {self.E}, A: {self.A}, I: {self.I}'


In [85]:
class structure:
	def __init__(self, df):
		self.nodes = {}
		self.elements = {}

		pos = 0
		n_nodes = df.iloc[pos, 0]
		pos = pos + 1

		for _ in range(1, n_nodes+1):
			self.nodes[df.iloc[pos, 0]] = node(df.iloc[pos, 1], df.iloc[pos, 2], [df.iloc[pos, 3], df.iloc[pos, 4], df.iloc[pos, 5]])
			pos += 1
		
		n_elements = df.iloc[pos, 0]
		pos = pos + 1

		for _ in range(1, n_elements+1):
			self.elements[df.iloc[pos, 0]] = element(self.nodes[df.iloc[pos, 1]], self.nodes[df.iloc[pos, 2]], df.iloc[pos, 3], df.iloc[pos, 4], df.iloc[pos, 5])
			pos += 1

	def __str__(self):
		s = 'Structure - nodes: \n'	
		for key, value in self.nodes.items():
			s += f'{key}: {value}\n'
		for key, value in self.elements.items():
			s += f'{key}: {value}\n'
		return s

In [86]:
s = structure(df)

print(s)

Structure - nodes: 
1: Node - x: 0.0, y: 0.0, constraints: [1.0, 1.0, 1.0]
2: Node - x: 0.0, y: 6.0, constraints: [0.0, 0.0, 0.0]
3: Node - x: 4.0, y: 6.0, constraints: [1.0, 1.0, 1.0]
1: Element - node1: Node - x: 0.0, y: 0.0, constraints: [1.0, 1.0, 1.0], node2: Node - x: 0.0, y: 6.0, constraints: [0.0, 0.0, 0.0], L: 6.0, E: 1.0, A: 1.0, I: 1.0
2: Element - node1: Node - x: 0.0, y: 6.0, constraints: [0.0, 0.0, 0.0], node2: Node - x: 4.0, y: 6.0, constraints: [1.0, 1.0, 1.0], L: 4.0, E: 1.0, A: 1.0, I: 1.0



In [87]:
def transform(data, log=False):
    nodes = {}
    supports = {}
    conec = []
    loads = {}

    pos = 0

    n_nodes = data.iloc[0, 0]
    pos = pos + 1

    for i in range(pos, pos + n_nodes):
        node_id = str(data.iloc[i, 0])
        nodes[node_id] = [data.iloc[i, 1], data.iloc[i, 2]]
        supports[node_id] = (data.iloc[i, 4], data.iloc[i, 5], data.iloc[i, 6])
        pos = pos + 1

    n_conec = data.iloc[pos, 0]
    pos = pos + 1

    for i in range(pos, pos + n_conec):
        conec.append([str(int(data.iloc[i, 1])), str(int(data.iloc[i, 2]))])
        pos = pos + 1

    n_loads = data.iloc[pos, 0]
    pos = pos + 1

    for i in range(pos, pos + n_loads):
        node_id = str(data.iloc[i, 0])
        if loads.get(node_id) is None:
            loads[node_id] = [data.iloc[i, 1], data.iloc[i, 2], data.iloc[i, 3]]
        else:
            loads[node_id][0] += data.iloc[i, 1]
            loads[node_id][1] += data.iloc[i, 2]
            loads[node_id][2] += data.iloc[i, 3]
        pos = pos + 1

    if log:
        print("Nodes: \n")
        print(nodes)
        print("\nSupports: \n")
        print(supports)
        print("\nConections: \n")
        print(conec)
        print("\nLoads: \n")
        print(loads)

    structure_data = {'nodes':nodes, 'supports':supports, 'conec':conec, 'loads':loads}
    # print(structure_data)

    return structure_data